This notebook gives the possiblity to download gene expression data grom GEO, normalizing it and giving an output table in .tsv format.

Example will be shown on Ulloa-Montoya GSE35640, GPL570 sample

# Import of python base packages

In [37]:
%load_ext autoreload
%matplotlib inline
%config IPCompleter.use_jedi = False

import pathlib
import subprocess
import logging
import os
import pandas as pd
import csv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from portraits.mapping import get_gs_for_probes_from_3col,get_expressions_for_gs


In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# Get the example data from GEO

If you want to use another sample please change the GSE and PLATFORM variables to your desired samples GSE and PLATFORM values in GEO correspondingly.

In [47]:
GSE = 'GSE35640'
PLATFORM = 'GPL570'
EXPRESSION_MATRIX = 'Test/expression.tsv'

Initialize and create a temporary directory for the CEL files

In [21]:
current_dir = pathlib.Path().parent.absolute()
dir_to_process = str(current_dir / 'TMPDIR')

In [22]:
if not os.path.exists(dir_to_process):
     os.mkdir(dir_to_process)

In [23]:
with open(os.devnull, "w") as f:
    subprocess.run([
        'wget',
        f'ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE35nnn/{GSE}/suppl/{GSE}_RAW.tar'
    ],stdout=f, stderr=subprocess.STDOUT );

In [24]:
subprocess.run([
    'tar',
    '-xf',
    f'{GSE}_RAW.tar',
    '-C', dir_to_process
])
subprocess.run([
    'rm',
    f'{GSE}_RAW.tar'
])

CompletedProcess(args=['rm', 'GSE35640_RAW.tar'], returncode=0)

In [25]:
os.listdir(dir_to_process)

['GSM872328_MAGE008_sample_1.CEL.gz',
 'GSM872329_MAGE008_sample_2.CEL.gz',
 'GSM872330_MAGE008_sample_3.CEL.gz',
 'GSM872331_MAGE008_sample_4.CEL.gz',
 'GSM872332_MAGE008_sample_5.CEL.gz',
 'GSM872333_MAGE008_sample_6.CEL.gz',
 'GSM872334_MAGE008_sample_7.CEL.gz',
 'GSM872335_MAGE008_sample_8.CEL.gz',
 'GSM872336_MAGE008_sample_9.CEL.gz',
 'GSM872337_MAGE008_sample_10.CEL.gz',
 'GSM872338_MAGE008_sample_11.CEL.gz',
 'GSM872339_MAGE008_sample_12.CEL.gz',
 'GSM872340_MAGE008_sample_13.CEL.gz',
 'GSM872341_MAGE008_sample_14.CEL.gz',
 'GSM872342_MAGE008_sample_15.CEL.gz',
 'GSM872343_MAGE008_sample_16.CEL.gz',
 'GSM872344_MAGE008_sample_17.CEL.gz',
 'GSM872345_MAGE008_sample_18.CEL.gz',
 'GSM872346_MAGE008_sample_19.CEL.gz',
 'GSM872347_MAGE008_sample_20.CEL.gz',
 'GSM872348_MAGE008_sample_21.CEL.gz',
 'GSM872349_MAGE008_sample_22.CEL.gz',
 'GSM872350_MAGE008_sample_23.CEL.gz',
 'GSM872351_MAGE008_sample_24.CEL.gz',
 'GSM872352_MAGE008_sample_25.CEL.gz',
 'GSM872353_MAGE008_sample_26.CEL.

## Extracting expression values from CEL file

For affy arrays without special probes, use RMA
For GPL570/GPL96, use gcrma

In [26]:
%%R

# Load required R packages
library(affy)
library(annotate)
library(gcrma)

R[write to console]: Loading required package: BiocGenerics

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


R[write to console]: Loading required package: Biobase

R[write to console]: Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


R[write to console]

In [27]:
%%R -i dir_to_process -o normalized_expression

# Bulk read cell files
raw_expression <- ReadAffy(celfile.path = dir_to_process)

# Normalize expression using RMA
rma_normalized <- gcrma(raw_expression)

# Retrieve expressions from dataset
normalized_expression <- as.data.frame(exprs(rma_normalized))

R[write to console]: 



Adjusting for optical effect.................................................................Done.
Computing affinities.Done.
Adjusting for non-specific binding.................................................................Done.
Normalizing
Calculating Expression


In [28]:
normalized_expression.head()

,GSM872328_MAGE008_sample_1.CEL.gz,GSM872329_MAGE008_sample_2.CEL.gz,GSM872330_MAGE008_sample_3.CEL.gz,GSM872331_MAGE008_sample_4.CEL.gz,GSM872332_MAGE008_sample_5.CEL.gz,GSM872333_MAGE008_sample_6.CEL.gz,GSM872334_MAGE008_sample_7.CEL.gz,GSM872335_MAGE008_sample_8.CEL.gz,GSM872336_MAGE008_sample_9.CEL.gz,GSM872337_MAGE008_sample_10.CEL.gz,...,GSM872383_MAGE008_sample_56.CEL.gz,GSM872384_MAGE008_sample_57.CEL.gz,GSM872385_MAGE008_sample_58.CEL.gz,GSM872386_MAGE008_sample_59.CEL.gz,GSM872387_MAGE008_sample_60.CEL.gz,GSM872388_MAGE008_sample_61.CEL.gz,GSM872389_MAGE008_sample_62.CEL.gz,GSM872390_MAGE008_sample_63.CEL.gz,GSM872391_MAGE008_sample_64.CEL.gz,GSM872392_MAGE008_sample_65.CEL.gz
1007_s_at,9.816090,6.474644,9.113994,7.044742,8.719946,4.852191,7.190953,8.636192,5.623183,7.433451,...,6.287400,7.017124,7.533525,7.042195,8.809050,8.198514,8.457781,7.530108,4.724734,9.222943
1053_at,5.664499,6.540040,5.569006,5.745150,4.690936,5.923728,5.700713,5.120923,5.394368,6.704893,...,5.992869,6.754348,5.943736,6.333925,6.063615,6.360003,6.073347,7.141974,7.191548,5.611221
117_at,8.394219,6.083483,6.587381,6.092671,3.411626,3.791668,5.667734,4.965991,4.475886,4.443469,...,3.296701,5.638913,3.205921,3.471334,3.682211,8.971476,4.343425,5.738228,4.807307,10.331267
121_at,2.390202,2.345001,2.390202,2.388071,2.390202,2.390202,2.454035,2.390202,3.128954,2.390202,...,2.320004,2.390202,2.411101,2.390202,2.390202,2.390202,2.380696,2.390202,2.882576,2.434246
1255_g_at,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,...,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.348901,2.223272,2.223272,2.223272


In [48]:
# Trim names to make the table more readable.

normalized_expression.columns = normalized_expression.columns.to_series().apply(lambda x: x.split('_')[0]).values
normalized_expression.head()

,GSM872328,GSM872329,GSM872330,GSM872331,GSM872332,GSM872333,GSM872334,GSM872335,GSM872336,GSM872337,...,GSM872383,GSM872384,GSM872385,GSM872386,GSM872387,GSM872388,GSM872389,GSM872390,GSM872391,GSM872392
1007_s_at,9.816090,6.474644,9.113994,7.044742,8.719946,4.852191,7.190953,8.636192,5.623183,7.433451,...,6.287400,7.017124,7.533525,7.042195,8.809050,8.198514,8.457781,7.530108,4.724734,9.222943
1053_at,5.664499,6.540040,5.569006,5.745150,4.690936,5.923728,5.700713,5.120923,5.394368,6.704893,...,5.992869,6.754348,5.943736,6.333925,6.063615,6.360003,6.073347,7.141974,7.191548,5.611221
117_at,8.394219,6.083483,6.587381,6.092671,3.411626,3.791668,5.667734,4.965991,4.475886,4.443469,...,3.296701,5.638913,3.205921,3.471334,3.682211,8.971476,4.343425,5.738228,4.807307,10.331267
121_at,2.390202,2.345001,2.390202,2.388071,2.390202,2.390202,2.454035,2.390202,3.128954,2.390202,...,2.320004,2.390202,2.411101,2.390202,2.390202,2.390202,2.380696,2.390202,2.882576,2.434246
1255_g_at,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,...,2.223272,2.223272,2.223272,2.223272,2.223272,2.223272,2.348901,2.223272,2.223272,2.223272


In [30]:
# Delete unnecessary files

subprocess.run(['rm',  '-r','TMPDIR/'])

CompletedProcess(args=['rm', '-r', 'TMPDIR/'], returncode=0)

## Converting probe ids to HUGO gene symbols

To download the SOFT file manually, follow these steps (we use the Ulloya-Montoya sample as an example):

* Go to https://www.ncbi.nlm.nih.gov/geo/
* Type the GLP platform number – GLP570 – in the search bar
* Go to https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GPL570
* Find the Annotation SOFT table at the bottom of the page
* Click the button to download the SOFT annotation table for your sample (GPL570.annot.gz in our example)
* Upload the file to the environment.

To download SOFT annotation table from jupyter notebook, change the PLATFORM value to appropriate GPL platform.

In [31]:
with open(os.devnull, "w") as f:
    subprocess.run([
        'wget',
        f'ftp://ftp.ncbi.nlm.nih.gov/geo/platforms/GPLnnn/{PLATFORM}/annot/{PLATFORM}.annot.gz'
    ], stdout=f, stderr=subprocess.STDOUT)

Once you’ve downloaded the SOFT file, extract the data from it. Unzip the SOFT file to get GPL570.annot.gz

In [32]:
subprocess.run(['gunzip', f'{PLATFORM}.annot.gz'])

CompletedProcess(args=['gunzip', 'GPL570.annot.gz'], returncode=0)

Remove the header from the SOFT file to avoid any problems in further processes

In [33]:
subprocess.run("sed '1,/^ID/d' GPL570.annot > GPL570.beheaded.annot", shell = True)

CompletedProcess(args="sed '1,/^ID/d' GPL570.annot > GPL570.beheaded.annot", returncode=0)

Turn the SOFT file into a 3-column file.<br>

1st column: probe id<br>
2nd column: gene symbol column (as is with '///')<br>
3rd column: entrez id (not needed for the study)

Subsetting SOFT file to have 3 columns 

In [38]:
gene_SOFT_annotations = pd.read_csv(f'{PLATFORM}.beheaded.annot', sep = '\t', header = None)
gene_SOFT_annotations = gene_SOFT_annotations.iloc[:, [0, 2, 3]]

# Rename columns
gene_SOFT_annotations = gene_SOFT_annotations.rename(columns = {2: 1, 3 : 2})
gene_SOFT_annotations.to_csv(f'{PLATFORM}.3col', sep = '\t',  index=False, quoting=csv.QUOTE_NONNUMERIC)

In [39]:
gene_SOFT_annotations.head()

,0,1,2
0,1007_s_at,MIR4640///DDR1,100616237///780
1,1053_at,RFC2,5982
2,117_at,HSPA6,3310
3,121_at,PAX8,7849
4,1255_g_at,GUCA1A,2978


Delete all unnecessary files

In [40]:
subprocess.run(['rm', f'{PLATFORM}.annot'])
subprocess.run(['rm', f'{PLATFORM}.beheaded.annot'])

CompletedProcess(args=['rm', 'GPL570.beheaded.annot'], returncode=0)

In [44]:
probes_gs_dict = get_gs_for_probes_from_3col(f'{PLATFORM}.3col', normalized_expression.index.tolist())

In [45]:
pd.Series(probes_gs_dict).head(10)

1007_s_at    [MIR4640, DDR1]
1053_at               [RFC2]
117_at               [HSPA6]
121_at                [PAX8]
1255_g_at           [GUCA1A]
1294_at      [MIR5193, UBA7]
1316_at               [THRA]
1320_at             [PTPN21]
1405_i_at             [CCL5]
1431_at             [CYP2E1]
dtype: object

In [49]:
series = pd.Series(probes_gs_dict)
annotated_expression = get_expressions_for_gs(series, normalized_expression, 'max').T.sort_index()

annotated_expression.to_csv(EXPRESSION_MATRIX, sep='\t', index=True)

NameError: name 'EXPRESSION_MATRIX' is not defined

In case you want to download your data from GEO change the GSE and PLATFORM to the corresponding values for your chosen data.

In [47]:
GSE = 'GSE35640'
PLATFORM = 'GPL570'

# Data preparation

Example will be shown on Ulloa-Montoya GSE35640, GPL570 sample

## Get the example data from GEO

Initialize and create a temporary directory for the CEL files

In [ ]:
current_dir = pathlib.Path().parent.absolute()
dir_to_process = str(current_dir / 'TMPDIR')

In [ ]:
if not os.path.exists(dir_to_process):
     os.mkdir(dir_to_process)

In [ ]:
with open(os.devnull, "w") as f:
    subprocess.run([
        'wget',
        f'ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE35nnn/{GSE}/suppl/{GSE}_RAW.tar'
    ],stdout=f, stderr=subprocess.STDOUT );

In [ ]:
subprocess.run([
    'tar',
    '-xf',
    f'{GSE}_RAW.tar',
    '-C', dir_to_process
])
subprocess.run([
    'rm',
    f'{GSE}_RAW.tar'
])

In [ ]:
os.listdir(dir_to_process)

## Extracting expression values from CEL file

For affy arrays without special probes, use RMA
For GPL570/GPL96, use gcrma

In [ ]:
%%R

# Load required R packages
library(affy)
library(annotate)
library(gcrma)

In [ ]:
%%R -i dir_to_process -o normalized_expression

# Bulk read cell files
raw_expression <- ReadAffy(celfile.path = dir_to_process)

# Normalize expression using RMA
rma_normalized <- gcrma(raw_expression)

# Retrieve expressions from dataset
normalized_expression <- as.data.frame(exprs(rma_normalized))

In [ ]:
normalized_expression.head()

In [ ]:
# Trim names to make the table more readable.

normalized_expression.columns = normalized_expression.columns.to_series().apply(lambda x: x.split('_')[0]).values
normalized_expression.head()

In [ ]:
# Delete unnecessary files

subprocess.run(['rm',  '-r','TMPDIR/'])

## Converting probe ids to HUGO gene symbols

To download the SOFT file manually, follow these steps (we use the Ulloya-Montoya sample as an example):

* Go to https://www.ncbi.nlm.nih.gov/geo/
* Type the GLP platform number – GLP570 – in the search bar
* Go to https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GPL570
* Find the Annotation SOFT table at the bottom of the page
* Click the button to download the SOFT annotation table for your sample (GPL570.annot.gz in our example)
* Upload the file to the environment.

To download SOFT annotation table from jupyter notebook, change the PLATFORM value to appropriate GPL platform.

In [ ]:
with open(os.devnull, "w") as f:
    subprocess.run([
        'wget',
        f'ftp://ftp.ncbi.nlm.nih.gov/geo/platforms/GPLnnn/{PLATFORM}/annot/{PLATFORM}.annot.gz'
    ], stdout=f, stderr=subprocess.STDOUT)

Once you’ve downloaded the SOFT file, extract the data from it. Unzip the SOFT file to get GPL570.annot.gz

In [ ]:
subprocess.run(['gunzip', f'{PLATFORM}.annot.gz'])

Remove the header from the SOFT file to avoid any problems in further processes

In [ ]:
subprocess.run("sed '1,/^ID/d' GPL570.annot > GPL570.beheaded.annot", shell = True)

Turn the SOFT file into a 3-column file.<br>

1st column: probe id<br>
2nd column: gene symbol column (as is with '///')<br>
3rd column: entrez id (not needed for the study)

Subsetting SOFT file to have 3 columns 

In [ ]:
gene_SOFT_annotations = pd.read_csv(f'{PLATFORM}.beheaded.annot', sep = '\t', header = None)
gene_SOFT_annotations = gene_SOFT_annotations.iloc[:, [0, 2, 3]]

# Rename columns
gene_SOFT_annotations = gene_SOFT_annotations.rename(columns = {2: 1, 3 : 2})
gene_SOFT_annotations.to_csv(f'{PLATFORM}.3col', sep = '\t',  index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
gene_SOFT_annotations.head()

Delete all unnecessary files

In [ ]:
subprocess.run(['rm', f'{PLATFORM}.annot'])
subprocess.run(['rm', f'{PLATFORM}.beheaded.annot'])

In [ ]:
probes_gs_dict = get_gs_for_probes_from_3col(f'{PLATFORM}.3col', normalized_expression.index.tolist())

In [ ]:
pd.Series(probes_gs_dict).head(10)

In [ ]:
series = pd.Series(probes_gs_dict)
annotated_expression = get_expressions_for_gs(series, normalized_expression, 'max').T.sort_index()

annotated_expression.to_csv(EXPRESSION_MATRIX, sep='\t', index=True)

# Quality check (QC)